### to break down the repo code: subset

In [ ]:
### things i want to learn more about: under the hood
## ml-ops
## python context object (with), defines logic for entering and exiting this context
## eg. open file, read file, close file; open model, read model, close model

### build concept classifier (rep for the concept)
gather feature representations for classifying concept
- occupation

In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)

from datasets import load_dataset


/common/home/users/x/xinrose.lin.2020/.conda/envs/fcs-exp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from nnsight import LanguageModel

#  model hyperparameters
# DEVICE = 'cuda:0'
DEVICE = "cpu"
model = LanguageModel('EleutherAI/pythia-70m-deduped', device_map=DEVICE, dispatch=True)
activation_dim = 512

In [3]:
# dataset hyperparameters
dataset = load_dataset("LabHC/bias_in_bios")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 257478
    })
    test: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 99069
    })
    dev: Dataset({
        features: ['hard_text', 'profession', 'gender'],
        num_rows: 39642
    })
})

In [5]:
dataset['train'][:5]

{'hard_text': ['He is also the project lead of and major contributor to the open source assembler/simulator "EASy68K." He earned a master’s degree in computer science from the University of Michigan-Dearborn, where he is also an adjunct instructor. Downloads/Updates',
  'She is able to assess, diagnose and treat minor illness conditions and exacerbations of some long term conditions. Her qualifications include Registered General Nurse, Bachelor of Nursing, Diploma in Health Science, Emergency Care Practitioner and Independent Nurse Prescribing.',
  'Prior to law school, Brittni graduated magna cum laude from DePaul University in 2011 with her Bachelor’s Degree in Psychology and Spanish. In 2014, she earned her law degree from Chicago-Kent College of Law. While at Chicago-Kent, Brittni was awarded two CALI Excellence for the Future Awards in both Legal Writing and for her seminar article regarding President Obama’s executive action, Deferred Action for Childhood Arrivals.',
  'He regula

In [19]:
import torch as t
import random

# dataset hyperparameters
dataset = load_dataset("LabHC/bias_in_bios")
profession_dict = {'professor' : 21, 'nurse' : 13}
male_prof = 'professor'
female_prof = 'nurse'

# data preparation hyperparameters
batch_size = 8
SEED = 42

# To fit on 24GB VRAM GPU, I set the next 2 default batch_sizes to 64
def get_data(train=True, ambiguous=True, batch_size=batch_size, seed=SEED):
    if train:
        data = dataset['train']
    else:
        data = dataset['test']
    if ambiguous:
        
        neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        n = min([len(neg), len(pos)])
        neg, pos = neg[:n], pos[:n]
        data = neg + pos
        labels = [0]*n + [1]*n
        idxs = list(range(2*n))
        random.Random(seed).shuffle(idxs)
        data, labels = [data[i] for i in idxs], [labels[i] for i in idxs]
        true_labels = spurious_labels = labels
    else:
        ## this dataset is used; extract from dataset if:
        ## neg neg: male profession, male gender (0)
        ## neg pos: male profession, female gender (1)
        ## pos neg: female profession, male gender (0)
        ## pos pos: female profession, female gender (1)
        neg_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        neg_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 1]
        pos_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 0]
        pos_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        n = min([len(neg_neg), len(neg_pos), len(pos_neg), len(pos_pos)])
        neg_neg, neg_pos, pos_neg, pos_pos = neg_neg[:n], neg_pos[:n], pos_neg[:n], pos_pos[:n]
        data = neg_neg + neg_pos + pos_neg + pos_pos
        true_labels     = [0]*n + [0]*n + [1]*n + [1]*n
        spurious_labels = [0]*n + [1]*n + [0]*n + [1]*n
        idxs = list(range(4*n))
        random.Random(seed).shuffle(idxs)
        data, true_labels, spurious_labels = [data[i] for i in idxs], [true_labels[i] for i in idxs], [spurious_labels[i] for i in idxs]

    batches = [
        (data[i:i+batch_size], t.tensor(true_labels[i:i+batch_size], device=DEVICE), t.tensor(spurious_labels[i:i+batch_size], device=DEVICE)) for i in range(0, len(data), batch_size)
    ]

    return batches

In [20]:
batches = get_data(train=False, ambiguous=False)
batches[:5]

[(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
   'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart a

In [21]:
print('batches: ', len(batches))
print('each batch: ', len(batches[0][0]))
## (text, profession, gender)

batches:  217
each batch:  8


In [ ]:
def get_subgroups(train=True, ambiguous=True, batch_size=128, seed=SEED):
    if train:
        data = dataset['train']
    else:
        data = dataset['test']
    if ambiguous:
        neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        neg_labels, pos_labels = (0, 0), (1, 1)
        subgroups = [(neg, neg_labels), (pos, pos_labels)]
    else:
        neg_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 0]
        neg_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[male_prof] and x['gender'] == 1]
        pos_neg = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 0]
        pos_pos = [x['hard_text'] for x in data if x['profession'] == profession_dict[female_prof] and x['gender'] == 1]
        neg_neg_labels, neg_pos_labels, pos_neg_labels, pos_pos_labels = (0, 0), (0, 1), (1, 0), (1, 1)
        
        subgroups = [(neg_neg, neg_neg_labels), (neg_pos, neg_pos_labels), (pos_neg, pos_neg_labels), (pos_pos, pos_pos_labels)]
    
    out = {}
    for data, label_profile in subgroups:
        out[label_profile] = []
        for i in range(0, len(data), batch_size):
            text = data[i:i+batch_size]
            out[label_profile].append(
                (
                    text,
                    t.tensor([label_profile[0]]*len(text), device=DEVICE),
                    t.tensor([label_profile[1]]*len(text), device=DEVICE)
                )
            )
    return out

#### classifier
- linear
- ARE: classifier is nonlinear?

In [22]:
import torch.nn as nn

# probe training hyperparameters

layer = 4 # model layer for attaching linear classification head

class Probe(nn.Module):
    def __init__(self, activation_dim):
        super().__init__()
        self.net = nn.Linear(activation_dim, 1, bias=True)

    def forward(self, x):
        logits = self.net(x).squeeze(-1)
        return logits

### from ARE repo
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, num_classes)

    # @autocast()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        #x = nn.functional.softmax(x, dim=1)
        return x

In [23]:
DEBUGGING = False

if DEBUGGING:
    tracer_kwargs = dict(scan=True, validate=True)
else:
    tracer_kwargs = dict(scan=False, validate=False)
    

def get_acts(text):
    with t.no_grad(): 
        with model.trace(text, **tracer_kwargs):
            attn_mask = model.input[1]['attention_mask']
            acts = model.gpt_neox.layers[layer].output[0]
            acts = acts * attn_mask[:, :, None]
            acts = acts.sum(1) / attn_mask.sum(1)[:, None]
            acts = acts.save()
        return acts.value
    

#### aside: nnsight trace context

In [76]:
## python context object (with), defines logic for entering and exiting this context
## eg. open file, read file, close file; open model, read model, close model

## model only executed at the end of tracing context 
## .input .output within context requires .save() to be accessed at the end of context

from nnsight import NNsight
from collections import OrderedDict

## ordered-dict: for accessing via keys
net = nn.Sequential(
    OrderedDict(
        [
            ('layer1', nn.Linear(5, 10)), 
            ('layer2', nn.Linear(10, 2))
    ]  
    )
).requires_grad_(False)

tiny_model = NNsight(net)

input = t.rand((1, 5))

with tiny_model.trace(input) as tracer:
    input = tiny_model.input.save()
    layer1 = tiny_model.layer1.output.save()
    ## can also log to save memory; if we dont need it after
    tracer.log('model out', tiny_model.output)

print(input.value)
print(layer1.value)


model out tensor([[-0.2079, -0.2004]])
tensor([[0.4059, 0.4403, 0.9451, 0.4350, 0.4141]])
tensor([[ 0.1934,  0.9809,  0.2468, -0.4346,  1.0432, -0.4203,  0.0243, -0.1003,
          0.2104, -0.1645]])


In [80]:
input.shape

torch.Size([1, 5])

In [72]:
print('tiny model', type(tiny_model))
print('layers', type(tiny_model.layer2)) ### envoy??

tiny model <class 'nnsight.models.NNsightModel.NNsight'>
layers <class 'nnsight.envoy.Envoy'>


In [67]:
## function, methods, operations can happen within context 
## will simply be executed when model is ran
## only values with .save() will be saved
## HOWEVER, since tracing context operates on intervention graph
## to apply a custom function, it needs to be part of the intervention graph

with tiny_model.trace(input):
    value = (tiny_model.layer1.output.sum() + tiny_model.layer2.output.sum()).save()
print(value)

tensor(-2.3806)


In [34]:
sample_text = batches[0][0][0]

In [85]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
        

In [84]:
tracer_kwargs = dict(scan=True, validate=True)
with t.no_grad(): 
    with model.trace(input, **tracer_kwargs) as tracer:
        tracer.log(model.output)

FakeTensor(..., size=(1, 2))
tensor([[-0.2079, -0.2004]])


In [73]:
with t.no_grad(): 
    with model.trace(sample_text, **tracer_kwargs):
        attn_mask = model.input[1]['attention_mask']
        print(attn_mask.shape)
        acts = model.gpt_neox.layers[layer].output[0]
        acts = acts * attn_mask[:, :, None]
        acts = acts.sum(1) / attn_mask.sum(1)[:, None]
        acts = acts.save()
    # return acts.value

IndexError: tuple index out of range

In [32]:
model.gpt_neox.layers[4]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXSdpaAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_dropout): Dropout(p=0.0, inplace=False)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
    (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
    (act): GELUActivation()
  )
)

In [94]:
batches[0]

(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
  'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and

In [99]:
import numpy as np 

new_batch = []
for batch in batches: 
    prof = batch[1].tolist()
    gender = batch[2].tolist()
    new_batch.append((batch[0], prof, gender))
    

In [101]:
new_batch[0]

(["She graduated with honors from University Of New Mexico School Of Medicine in 1999. Having more than 17 years of diverse experiences, especially in CERTIFIED NURSE MIDWIFE, Cristy Janese Walsh affiliates with many hospitals including Plains Regional Medical Center, Roosevelt General Hospital, and cooperates with other doctors and specialists in medical group Women's Medical Center Llp. Call Cristy Janese Walsh on phone number (575) 762-8055 for more information and advises or to book an appointment.",
  'He currently works in the heart transplant and heart failure programs. He has experience with newborn, pediatric, and adult complex congenital heart disease, cardiomyopathy, and transplant. He worked for over 10 years in the Pediatric & Cardiothoracic ICU. He also is a Clinical Nursing Instructor. He is an active member of the American Association of Critical-Care Nurses (AACN), the National Association of Transplant Professionals (NATCO), and the International Society for Heart and

In [102]:
import json
# File path to save the JSON file
file_path = 'exp_redo/batched_data.json'

# Writing list to JSON file
with open(file_path, 'w') as json_file:
    json.dump(new_batch, json_file)

print(f"List saved to {file_path}")

List saved to exp_redo/batched_data.json


In [ ]:
def train_probe(get_acts, label_idx=0, batches=get_data(), lr=1e-2, epochs=1, dim=512, seed=SEED):
    t.manual_seed(seed)
    probe = Probe(dim).to(DEVICE)
    optimizer = t.optim.AdamW(probe.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    losses = []
    for epoch in range(epochs):
        for batch in batches:
            
            text = batch[0]
            labels = batch[label_idx+1] 
            acts = get_acts(text)
            logits = probe(acts)
            loss = criterion(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

    return probe, losses

In [ ]:
def test_probe(probe, get_acts, label_idx=0, batches=get_data(train=False), seed=SEED):
    with t.no_grad():
        corrects = []

        for batch in batches:
            text = batch[0]
            labels = batch[label_idx+1]
            acts = get_acts(text)
            logits = probe(acts)
            preds = (logits > 0.0).long()
            corrects.append((preds == labels).float())
        return t.cat(corrects).mean().item()

### SAE latents of model
- load dictionaries of model
- train dictionaries of model

### build casual graph for concept classifier
- approximate causal effect of the 
- find the most causally relevant latent features for classification performance

### edit / inform ReFT based on the graph? 
- 